# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [40]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [41]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [42]:
# frequency table for prestige and whether or not someone was admitted
print pd.crosstab(index=df["prestige"],  # Make a crosstab
                              columns="count")      # Name the count column



col_0     count
prestige       
1.0          61
2.0         148
3.0         121
4.0          67


In [43]:
table1 = pd.crosstab(df["prestige"], 
                            columns=df['admit'])
print table1

admit      0   1
prestige        
1.0       28  33
2.0       95  53
3.0       93  28
4.0       55  12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [44]:
df = df.join(pd.get_dummies(df['prestige'], prefix = 'prestige_'))
df.head()


,admit,gre,gpa,prestige,prestige__1.0,prestige__2.0,prestige__3.0,prestige__4.0
0,0,380.0,3.61,3.0,0.0,0.0,1.0,0.0
1,1,660.0,3.67,3.0,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,1.0,0.0,0.0,0.0
3,1,640.0,3.19,4.0,0.0,0.0,0.0,1.0
4,0,520.0,2.93,4.0,0.0,0.0,0.0,1.0


In [45]:
df.shape

(397, 8)

#### 2.2 When modeling our class variables, how many do we need? 



Answer: seven

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [50]:
cols_to_keep = ['admit', 'gre', 'gpa','prestige__1.0','prestige__2.0','prestige__3.0']
handCalc = df[cols_to_keep]
print handCalc.head()

   admit    gre   gpa  prestige__1.0  prestige__2.0  prestige__3.0
0      0  380.0  3.61            0.0            0.0            1.0
1      1  660.0  3.67            0.0            0.0            1.0
2      1  800.0  4.00            1.0            0.0            0.0
3      1  640.0  3.19            0.0            0.0            0.0
4      0  520.0  2.93            0.0            0.0            0.0


In [51]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted

table2 = pd.crosstab(df["prestige__1.0"], 
                            columns=df['admit'])
print table2

admit            0   1
prestige__1.0         
0.0            243  93
1.0             28  33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [55]:
p = float(33)/(28+33)

print float(p)/(1-p)

1.17857142857


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [56]:
p2 = float(93)/(243+93)
print float(p2)/(1-p2)

0.382716049383


#### 3.3 Calculate the odds ratio

In [59]:
print  p/p2

1.95452141724


#### 3.4 Write this finding in a sentenance: 

Answer: the odd of those who attend ranked 1 college is 1.95 time of the odd of those who do not

#### 3.5 Print the cross tab for prestige_4

In [60]:
table3 = pd.crosstab(df["prestige__4.0"], 
                            columns=df['admit'])
print table3

admit            0    1
prestige__4.0          
0.0            216  114
1.0             55   12


#### 3.6 Calculate the OR 

In [61]:
p1 = float(12)/(55+12)
Odd1 = p1/(1-p1)
P2 = float(114)/(216+114)
Odd2 = p2/(1-p2)
OR = Odd1/Odd2
print OR

0.57008797654


#### 3.7 Write this finding in a sentence

Answer:the odd of those who attend ranked 4 college is 0.57 times of those who do not.

## Part 4. Analysis

In [62]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa','prestige__2.0','prestige__3.0', 'prestige__4.0']
data = df[cols_to_keep]
print data.head()

   admit    gre   gpa  prestige__2.0  prestige__3.0  prestige__4.0
0      0  380.0  3.61            0.0            1.0            0.0
1      1  660.0  3.67            0.0            1.0            0.0
2      1  800.0  4.00            0.0            0.0            0.0
3      1  640.0  3.19            0.0            0.0            1.0
4      0  520.0  2.93            0.0            0.0            1.0


In [74]:
data.shape

(397, 7)

We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [63]:
# manually add the intercept
data['intercept'] = 1.0
data.head()

/Users/weiwei/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,admit,gre,gpa,prestige__2.0,prestige__3.0,prestige__4.0,intercept
0,0,380.0,3.61,0.0,1.0,0.0,1.0
1,1,660.0,3.67,0.0,1.0,0.0,1.0
2,1,800.0,4.00,0.0,0.0,0.0,1.0
3,1,640.0,3.19,0.0,0.0,1.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0,1.0


#### 4.1 Set the covariates to a variable called train_cols

In [67]:
train_cols = ['admit', 'gre', 'gpa','prestige__2.0','prestige__3.0', 'prestige__4.0']

#### 4.2 Fit the model

In [75]:
from sklearn.linear_model import LogisticRegression
lm = LogisticRegression()
lm.fit(data[['admit', 'gre', 'gpa','prestige__2.0','prestige__3.0', 'prestige__4.0']], df['admit'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

#### 4.3 Print the summary results

In [76]:
print lm.coef_
print lm.intercept_
print df.admit.mean()


[[  6.50170462e+00  -6.58235845e-04  -6.30389345e-01  -2.59013281e-01
   -4.55822003e-01  -5.55093626e-01]]
[-0.72978676]
0.317380352645


#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [77]:
print np.exp(lm.coef_)
print np.exp(lm.intercept_)
print df.admit.mean()

[[  6.66276412e+02   9.99341981e-01   5.32384479e-01   7.71812772e-01
    6.33926664e-01   5.74018516e-01]]
[ 0.48201176]
0.317380352645


In [ ]:
# not sure how to do

#### 4.5 Interpret the OR of Prestige_2

Answer: 

#### 4.6 Interpret the OR of GPA

Answer: 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [78]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [79]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [ ]:
# recreate the dummy variables

# keep only what we need for making predictions


#### 5.2 Make predictions on the enumerated dataset

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.